# How to: Extracting EMIT Spectra using a Shapefile/GeoJSON

**Summary**  

In this notebook we will open a netCDF4 file from the Earth Surface Minteral Dust Source Investigation (EMIT) as an `xarray.Dataset`. We will then extract extract or clip to an area using a `.geojson` file (will also work with shapefile). The workflows outlined here will work with reflectance L2A or radiance L1B data.

**Requirements:**
+ A NASA [Earthdata Login](https://urs.earthdata.nasa.gov/) account is required to download EMIT data   
+ Selected the `emit_tutorials` environment as the kernel for this notebook.
  + For instructions on setting up the environment, follow the the `setup_instructions.md` included in the `/setup/` folder of the repository.  

**Learning Objectives**  
- How to open and EMIT Dataset as an `xarray.Dataset`
- How to extract values or clip an EMIT dataset to a region of interest
- How to write a new netCDF4 output using the clipped data

---

Import the required Python libraries.

In [1]:
# Import Packages
import os
import earthaccess
import xarray as xr
from osgeo import gdal
import rasterio as rio
import rioxarray as rxr
import hvplot.xarray
import hvplot.pandas
import holoviews as hv
import geopandas as gp
import sys
sys.path.append('../modules/')
from emit_tools import emit_xarray

Login to your NASA Earthdata account and create a `.netrc` file using the `login` function from the `earthaccess` library. If you do not have an Earthdata Account, you can create one [here](https://urs.earthdata.nasa.gov/home). 

In [2]:
earthaccess.login(persist=True)

EARTHDATA_USERNAME and EARTHDATA_PASSWORD are not set in the current environment, try setting them or use a different strategy (netrc, interactive)
You're now authenticated with NASA Earthdata Login
Using token with expiration date: 07/08/2023
Using .netrc file for EDL


For this notebook we will download the files necessary using `earthaccess`. You can also access the data in place or stream it, but this can slow due to the file sizes. Provide a URL for an EMIT L2A Reflectance granule.

In [3]:
url = 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20220903T163129_2224611_012/EMIT_L2A_RFL_001_20220903T163129_2224611_012.nc'

Get an HTTPS Session using your earthdata login, set a local path to save the file, and download the granule asset - This may take a while, the reflectance file is approximately 1.8 GB. 

In [4]:
# Get Https Session using Earthdata Login Info
fs = earthaccess.get_fsspec_https_session()
# Retrieve granule asset ID from URL (to maintain existing naming convention)
granule_asset_id = url.split('/')[-1]
# Define Local Filepath
fp = f'../../data/{granule_asset_id}'
# Download the Granule Asset if it doesn't exist
if not os.path.isfile(fp):
    fs.download(url, fp)

Open the file downloaded and defined as `fp`. To do this, we will use the `emit_xarray` function from the `emit_tools` module. This module contains a few helpful functions that can be used with EMIT data.

In [5]:
ds = emit_xarray(fp, ortho=True)
ds

<xarray.Dataset>
Dimensions:           (latitude: 2009, longitude: 2353, wavelengths: 285)
Coordinates:
  * latitude          (latitude) float64 -39.31 -39.31 -39.31 ... -40.4 -40.4
  * longitude         (longitude) float64 -62.51 -62.51 -62.51 ... -61.24 -61.24
  * wavelengths       (wavelengths) float32 381.0 388.4 ... 2.486e+03 2.493e+03
    good_wavelengths  (wavelengths) float32 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    fwhm              (wavelengths) float32 8.415 8.415 8.415 ... 8.807 8.809
    elev              (latitude, longitude) float32 nan nan nan ... nan nan nan
    spatial_ref       int32 0
Data variables:
    reflectance       (latitude, longitude, wavelengths) float32 nan nan ... nan
Attributes: (12/40)
    ncei_template_version:             NCEI_NetCDF_Swath_Template_v2.0
    summary:                           The Earth Surface Mineral Dust Source ...
    keywords:                          Imaging Spectroscopy, minerals, EMIT, ...
    Conventions:                       CF-1.63
    sensor:                            EMIT (Earth Surface Mineral Dust Sourc...
    instrument:                        EMIT
    ...                                ...
    spatial_ref:                       GEOGCS["WGS 84",DATUM["WGS_1984",SPHER...
    geotransform:                      [-6.25120945e+01  5.42232520e-04 -0.00...
    day_night_flag:                    Day
    title:                             EMIT L2A Estimated Surface Reflectance...
    granule_id:                        EMIT_L2A_RFL_001_20220903T163129_22246...
    Orthorectified:                    True

Using the `read_file()` function from `geopandas`, read in the `.geojson` file containing the polygon you wish to extract.

In [6]:
shape = gp.read_file('../../data/isla_gaviota.geojson')
shape

,geometry
0,"POLYGON ((-62.14758 -39.88951, -62.16900 -39.8..."


Now plot the polygon we've loaded overlayed on a plot of the dataset.

In [7]:
ds.sel(wavelengths=800,method='nearest').hvplot.image(cmap='greys', frame_width=500, rasterize=True)*shape.hvplot(color='#d95f02',aspect='equal', alpha=0.5)

:DynamicMap   []
   :Overlay
      .Image.I    :Image   [longitude,latitude]   (reflectance)
      .Polygons.I :Polygons   [x,y]

Use the `clip` function from `rasterio` to clip the dataset to polygons from the `geopandas.geodataframe`. Setting `all_touched` to `True` will include pixels that intersected with the edges of the polygon. 

In [8]:
clipped = ds.rio.clip(shape.geometry.values,shape.crs, all_touched=True)
clipped

<xarray.Dataset>
Dimensions:           (latitude: 140, longitude: 164, wavelengths: 285)
Coordinates:
  * latitude          (latitude) float64 -39.88 -39.88 -39.88 ... -39.95 -39.95
  * longitude         (longitude) float64 -62.2 -62.2 -62.2 ... -62.12 -62.12
  * wavelengths       (wavelengths) float32 381.0 388.4 ... 2.486e+03 2.493e+03
    good_wavelengths  (wavelengths) float32 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    fwhm              (wavelengths) float32 8.415 8.415 8.415 ... 8.807 8.809
    elev              (latitude, longitude) float32 12.89 12.89 ... 12.91 12.91
    spatial_ref       int32 0
Data variables:
    reflectance       (latitude, longitude, wavelengths) float32 nan nan ... nan
Attributes: (12/40)
    ncei_template_version:             NCEI_NetCDF_Swath_Template_v2.0
    summary:                           The Earth Surface Mineral Dust Source ...
    keywords:                          Imaging Spectroscopy, minerals, EMIT, ...
    Conventions:                       CF-1.63
    sensor:                            EMIT (Earth Surface Mineral Dust Sourc...
    instrument:                        EMIT
    ...                                ...
    spatial_ref:                       GEOGCS["WGS 84",DATUM["WGS_1984",SPHER...
    geotransform:                      [-6.25120945e+01  5.42232520e-04 -0.00...
    day_night_flag:                    Day
    title:                             EMIT L2A Estimated Surface Reflectance...
    granule_id:                        EMIT_L2A_RFL_001_20220903T163129_22246...
    Orthorectified:                    True

To view the clipped image, select a band from the `clipped` dataset and plot it spatially.

In [9]:
clipped.sel(wavelengths=800,method='nearest').hvplot.image(cmap='viridis', aspect = 'equal', frame_width=500, rasterize=True)

:DynamicMap   []
   :Image   [longitude,latitude]   (reflectance)

Now we can save the clipped `xarray.Dataset` as a netCDF4 output that can be reopened using the `xarray.open_dataset` function. 

In [10]:
clipped.to_netcdf('../../data/clipped_data.nc')
# Example for Opening 
#ds = xr.open_dataset('../../data/clipped_data.nc')

---

## Contact Info:  

Email: LPDAAC@usgs.gov  
Voice: +1-866-573-3222  
Organization: Land Processes Distributed Active Archive Center (LP DAAC)¹  
Website: <https://lpdaac.usgs.gov/>  
Date last modified: 06-30-2023  

¹Work performed under USGS contract G15PD00467 for NASA contract NNG14HH33I.  